In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
import torch.nn.functional as F

# 定义 SparseDenoisingAutoencoder 类
class SparseDenoisingAutoencoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(SparseDenoisingAutoencoder, self).__init__()
        self.encoder = nn.Linear(input_size, hidden_size)
        self.decoder = nn.Linear(hidden_size, input_size)

    def forward(self, x):
        encoded = F.relu(self.encoder(x))
        decoded = F.relu(self.decoder(encoded))
        return decoded

class StackedSparseDenoisingAutoencoder(nn.Module):
    def __init__(self, input_size, hidden_sizes):
        super(StackedSparseDenoisingAutoencoder, self).__init__()
        layers = []
        sizes = [input_size] + hidden_sizes
        for i in range(len(hidden_sizes)):
            layers.append(SparseDenoisingAutoencoder(sizes[i], sizes[i + 1]))
        self.layers = nn.ModuleList(layers)

    def forward(self, x):
        #x = x.view(x.size(0), -1)  # 将数据展平为(batch_size, input_size)形状
        for layer in self.layers:
            x = layer(x)
        return x

# 超参数
input_size = 400
hidden_sizes = [256, 128, 64]
num_epochs = 100
batch_size = 128
learning_rate = 0.001
noise_factor = 0.5

# 加载数据
data = np.load("SpectrumData.npy", allow_pickle=True)
data = data.T
# 归一化
mm = MinMaxScaler()
data = mm.fit_transform(data)
# 创建数据加载器
data = torch.tensor(data, dtype=torch.float32)
dataset = TensorDataset(data)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 初始化模型、损失函数和优化器
model = StackedSparseDenoisingAutoencoder(input_size, hidden_sizes)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 训练模型
for epoch in range(num_epochs):
    for batch_data in data_loader:
        batch_data = batch_data[0]
        noisy_data = batch_data + noise_factor * torch.randn(batch_data.size())

        # 前向传播
        outputs = model(noisy_data)

        # 计算损失
        loss = criterion(outputs, batch_data)

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 输出每轮训练结果
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch + 1, num_epochs, loss.item()))

print("Training completed.")


RuntimeError: mat1 and mat2 shapes cannot be multiplied (128x400 and 256x128)